# Importing libraries and datasets

In [105]:
import pandas as pd

In [147]:
cl_cat = pd.read_csv('client_categories.csv', sep=';')
cl_lst = pd.read_csv('clients_last_2_fixed.csv', sep=';')
transac = pd.read_csv('transac_137k.csv', index_col = 0)
mcc = pd.read_excel('MCC_last.xlsx')

In [148]:
transac.head()

,purchdate,amount,mcc,mrchcity,mrchname,cnum
202,2019-12-04 00:00:00,1240000.0,Clothes,MOSKVA,JHZ X994I1,0SGOOD
247,2019-10-15 00:00:00,1125573.0,Auto,GOVOROVO,XXX CWH7E12A HJ,0SXGSG
266,2019-11-18 00:00:00,1081005.0,Supermarkets,MOSCOW,ZLJC1-TJ1C1D7 ZX87,0FFNNS
352,2019-12-17 00:00:00,1000000.0,Legal and insurance,MOSKVA,XXX K3 J749914K1D H491,0XCONY
438,2019-11-29 00:00:00,910000.0,Auto,MYTISHHI,K1T1J5-0-L 7-C K01J,0GSCHC


In [149]:
cl_lst.head()

,cnum_,categorycode,gender,age,married_,residenttype
0,0CCCDO,81,M,32,not_married,R
1,0CCCFO,70,F,42,not_married,R
2,0CCCGC,50,F,33,married,R
3,0CCCGG,70,M,58,not_married,R
4,0CCCGO,60,M,38,married,R


Посмотрим на категории товаров

In [150]:
mcc['category'].value_counts()

Tourism                      641
Other                         43
Maintenance and building      28
Medical                       20
Auto                          18
Clothes                       18
Services                      15
Cultural life                 15
Legal and insurance           14
Active rest                   11
Design, print                 11
Furniture, home                9
Supermarkets                   8
Soft                           7
Sport                          6
Kids                           6
Money                          6
Postal services, delivery      5
Education                      5
Payments                       4
Digital goods                  4
Providers                      4
Discount                       4
Railway and sea transport      3
Gift, hobby                    3
Parking&Fuel                   3
Pet                            2
Beauty                         2
Alcohol and tobacco            2
Local and autotransport        2
Restaurant

# Обработка данных

In [151]:
cats = {}
for cat in mcc['category'].value_counts().index:
    cats[cat] = list(mcc.loc[mcc.category == cat]['mcc'].values)

In [152]:
mcc_list = transac['mcc'].value_counts().index
for cat_num in mcc_list:
    for cat in mcc['category'].value_counts().index:
        if cat_num in cats[cat]:
            transac.loc[transac.mcc == cat_num, 'mcc'] = cat

In [153]:
col = transac['mcc'].value_counts().index
for name in col:
    transac[name] = 0
    transac[name] = transac.loc[transac.mcc == name]['amount'] 
transac.fillna(0, inplace = True)

In [154]:
transac['Money'] = transac['Money']/1000

Получим датасет, где видны общие затраты на категорию товаров для каждого клиента

In [155]:
transac = transac.groupby(['cnum']).sum().iloc[:,4:]
transac.head()

,Supermarkets,Tourism,Medical,Payments,"Furniture, home",Restaurants,Beauty,Auto,Soft,Sport,...,3381,7277,5111,7296,5963,3503,3068,3579,780,5815
cnum,,,,,,,,,,,,,,,,,,,,,
0CCCDO,0.0,0.0,0.0,0.0,0.0,0.0,15506.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0CCCFO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0CCCGC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0CCCGG,8442.3,0.0,0.0,6000.0,0.0,0.0,0.0,73132.75,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0CCCGO,0.0,0.0,0.0,5500.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Обработаем датасет с клиентами

In [156]:
cl = cl_lst.join(pd.get_dummies(cl_lst['gender']))
cl = cl.join(pd.get_dummies(cl_lst['married_']))
cl = cl.join(pd.get_dummies(cl_lst['residenttype']))

In [157]:
count = ['gender','married_','residenttype']
cl.drop(count, axis=1, inplace=True)

In [158]:
cl.set_index('cnum_', inplace=True)
cl.head()

,categorycode,age,F,M,married,not_married,N,R
cnum_,,,,,,,,
0CCCDO,81,32,0,1,0,1,0,1
0CCCFO,70,42,1,0,0,1,0,1
0CCCGC,50,33,1,0,1,0,0,1
0CCCGG,70,58,0,1,0,1,0,1
0CCCGO,60,38,0,1,1,0,0,1


In [159]:
transac['max_group'] = [transac.columns[i][0] for i in transac.values == transac.max(axis=1)[:,None]]

In [160]:
transac.head()

,Supermarkets,Tourism,Medical,Payments,"Furniture, home",Restaurants,Beauty,Auto,Soft,Sport,...,7277,5111,7296,5963,3503,3068,3579,780,5815,max_group
cnum,,,,,,,,,,,,,,,,,,,,,
0CCCDO,0.0,0.0,0.0,0.0,0.0,0.0,15506.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Beauty
0CCCFO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Supermarkets
0CCCGC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Supermarkets
0CCCGG,8442.3,0.0,0.0,6000.0,0.0,0.0,0.0,73132.75,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Auto
0CCCGO,0.0,0.0,0.0,5500.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alcohol and tobacco


In [161]:
cl = cl.join(transac['max_group']).dropna()

In [162]:
cats = mcc['category'].value_counts().index
cl['mcc'] = 0
for i in range(len(cats)):
    cl.loc[cl.max_group == cats[i], 'mcc'] = i

In [163]:
cl.drop('max_group', axis=1, inplace=True)
cl.head()

,categorycode,age,F,M,married,not_married,N,R,mcc
cnum_,,,,,,,,,
0CCCDO,81,32,0,1,0,1,0,1,27
0CCCFO,70,42,1,0,0,1,0,1,12
0CCCGC,50,33,1,0,1,0,0,1,12
0CCCGG,70,58,0,1,0,1,0,1,4
0CCCGO,60,38,0,1,1,0,0,1,28


In [164]:
X = cl.iloc[:,:-1]
y = cl.iloc[:,-1:]

In [168]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# RandomForest

In [177]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth = 15)
rfc.fit(X_train, y_train)

<ipython-input-177-fbc60d7f8599>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)


RandomForestClassifier(max_depth=15)

In [178]:
rfc_pred = rfc.predict(X_test)
pd.DataFrame(rfc_pred)[0].value_counts()

12    2286
0       55
3       24
4       22
13      17
11      16
19      14
30      10
7        7
27       7
14       6
15       4
22       4
18       3
28       2
2        2
17       1
23       1
21       1
Name: 0, dtype: int64

In [179]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, rfc_pred)

0.45527800161160353